gives you tools to stear your investigations in the right direction whenever your automatice retraining pipeline looses capacity to deliver ever-better performing model versions.

some circumstances could legitimate to include preprocessing to hyperparam tuning (for instance, to dynamically evaluate different values for "num_feature1/buckets_count" )

in some situations, model blessing can also be conditionnal on carateristics such as "inference latency".

<hr />

# setup

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
!which python

In [ ]:
import os, sys, json, itertools

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# WandB API key
from dotenv import load_dotenv
load_dotenv("../.env")

In [ ]:
import sys
os.environ['conda_env'] = os.path.basename(sys.prefix)
os.environ['conda_env']

In [ ]:
mf_flow_name = 'LightGbmHpCvWandbFlow'
mf_flow_name

In [ ]:
from IPython import get_ipython

def reset_jupyter_history():
    """
    reset cell numbering
    """

    import os
    import sqlite3
    from traitlets.config import Config
    from IPython.core.history import HistoryManager
    
    ip = get_ipython()
    
    if ip is not None:
        # Disable history temporarily
        if hasattr(ip, 'history_manager'):
            ip.history_manager.enabled = False
        
        # Get the path to the history database
        hist_file = os.path.expanduser('~/.ipython/profile_default/history.sqlite')
        if hasattr(ip, 'history_manager') and hasattr(ip.history_manager, 'hist_file'):
            hist_file = ip.history_manager.hist_file
        
        if os.path.exists(hist_file):
            # Close any existing connection
            if hasattr(ip, 'history_manager') and hasattr(ip.history_manager, 'db') and ip.history_manager.db is not None:
                ip.history_manager.db.close()
            
            # Truncate the database
            conn = sqlite3.connect(hist_file)
            conn.execute("DELETE FROM history")
            conn.execute("DELETE FROM sqlite_sequence WHERE name='history'")
            conn.commit()
            conn.close()
        
        # Create a new Config object
        c = Config()
        c.HistoryManager.hist_file = hist_file
        
        # Create a new HistoryManager instance
        new_history_manager = HistoryManager(shell=ip, config=c)
        
        # Assign the new history manager to the shell
        ip.history_manager = new_history_manager
        
        # Enable history
        ip.history_manager.enabled = True

        ip.kernel.shell.execution_count = 0
        
        print("Jupyter history has been reset.")
    else:
        print("This function must be run within a Jupyter notebook.")

reset_jupyter_history()
get_ipython().kernel.shell.execution_count

<hr />

# Generate data

In [ ]:
from retrain_pipelines.dataset import DatasetType, pseudo_random_generate

num_samples = 1_500 # 10_000 # 30 # 500 # 
data = pseudo_random_generate(DatasetType.TABULAR_REGRESSION, num_samples)

print(data.head())
#data.to_csv(os.path.realpath(os.path.join('..', 'data', 'synthetic_classif_tab_data_continuous.csv')), index=False)

In [ ]:
#from sklearn.preprocessing import OrdinalEncoder
#
#os.environ["ml_pipeline_type"] = "lightgbm_regress_tempo"
#
#from retrain_pipelines.model import preprocess_data_fct
#
#encoder = OrdinalEncoder()
#pp_data = preprocess_data_fct(
#    data, encoder,
#    buckets={'num_feature1': [-1.002, -0.6, -0.19999999999999996, 0.20000000000000018, 0.6000000000000001, 1.0],
#             'num_feature2': 15},
#    is_training=True)
#
#del os.environ["ml_pipeline_type"]
#display(pp_data)
#display(encoder)

In [ ]:
from retrain_pipelines.dataset import features_desc
pd.DataFrame([features_desc(data)])

In [ ]:
from retrain_pipelines.dataset import features_distri_plot
display(features_distri_plot(data))

In [ ]:
from retrain_pipelines.dataset.features_dependencies import dataset_to_heatmap_fig
heatmap_fig, ax = dataset_to_heatmap_fig(data)
heatmap_fig.patch.set_facecolor((1.0, 1.0, 1.0, 0.6))
display(heatmap_fig)

In [ ]:
from io import BytesIO
bytes_io_obj = BytesIO()
heatmap_fig.savefig(bytes_io_obj, format='png',
                    bbox_inches='tight', pad_inches=0.05)

bytes_io_obj.seek(0)
with open('figure.png', 'wb') as f:
    f.write(bytes_io_obj.getvalue())

<hr />

# Metaflow Run

Chosse which domain shall be considered for the HP tuning grid search&nbsp;:

In [ ]:
pipeline_hp_grid = {
    "boosting_type": ["gbdt"],
    "num_leaves": [10],
    "learning_rate": [0.01],
    "n_estimators": [2],
}
os.environ['pipeline_hp_grid'] = str(json.dumps(pipeline_hp_grid)).replace("\n", "")
print(os.environ['pipeline_hp_grid'])

In [ ]:
pipeline_hp_grid = {
    "boosting_type": ["gbdt"],
    "num_leaves": [75, 100, 125],
    "learning_rate": [0.01],
    "n_estimators": [50],
    "lambda_l1": [0, 0.05],
    "lambda_l2": [0.1, 0.2, 0.3],
    "bagging_fraction": [1, 0.95],
}
os.environ['pipeline_hp_grid'] = str(json.dumps(model_hp_grid))
print(os.environ['pipeline_hp_grid'])
len([dict(zip(model_hp_grid.keys(), v))
     for v in itertools.product(*model_hp_grid.values())])

## Run flow

In [ ]:
%load_ext retrain_pipelines.local_launcher_magic

In [ ]:
%retrain_pipelines_local retraining_pipeline.py run --help

In [ ]:
%retrain_pipelines_local retraining_pipeline.py run \
    --data_file "../data/synthetic_classif_tab_data_continuous.csv" \
    --buckets_param '{"num_feature1": 100, "num_feature2": 50}' \
    --pipeline_hp_grid "${pipeline_hp_grid}" \
    --cv_folds 2 \
    --wandb_run_mode disabled
#\
    # --max-workers 4 \
    # --dask_partitions 4 \
    #--pipeline_card_artifacts_path "." \
    #--preprocess_artifacts_path "." \

In [ ]:
%retrain_pipelines_local retraining_pipeline.py resume pipeline_card

# Inspectors

We can programatically interact with the Metaflow service using the `metaflow`python package. To connect the package with our self-hosted metaflow service, we simply need to set a couple environment variables before importing it&nbsp;:

## local Metaflow SDK

In [ ]:
from retrain_pipelines.frameworks import local_metaflow as metaflow

In [ ]:
metaflow.Task("LightGbmHpCvWandbFlow/897/pipeline_card/27936",  attempt=0)['encoder'].data.__dict__

Inspectors are convenience methods that abstract away some of the logic to get access to Metaflow pipeline-run artifacts.

## local custom card explorer

In [ ]:
from retrain_pipelines.inspectors import browse_local_pipeline_card

In [ ]:
#browse_local_pipeline_card(mf_flow_name, mf_run_id=633, verbose=True)
browse_local_pipeline_card(mf_flow_name)
#browse_local_pipeline_card(mf_run_id=797, verbose=True)

<hr />

## WandB

In [ ]:
from retrain_pipelines.inspectors import get_execution_source_code

In [ ]:
help(get_execution_source_code)

In [ ]:
artifact_files = get_execution_source_code(mf_run_id=978)

for file in artifact_files:
    print(f" - {file.name} {file.url}")
    # file.download(replace=True)

In [ ]:
from retrain_pipelines.inspectors import explore_source_code
# download and open file explorer
explore_source_code(mf_run_id=978)

<hr />

## hp_cv_inspector

The herein retraining pipeline rely on Dask for data-parallel training. Each Cross-Validation fold of each set of hyperparameter value is trained using a subset of the dataset, parallelized accross workers.

We look into pipeline runs from the perspective of detailed training logs during hyperparameter tuning.

First, focusing on the best-performing set of hyperparameters values&nbsp;:

In [ ]:
from inspectors import plot_run_cv_history
plot_run_cv_history(best_cv=True) #mf_run_id=432, 

Now, looking at all sets of hyperparameter values evaluated&nbsp;:

In [ ]:
from inspectors import plot_run_all_cv_tasks
plot_run_all_cv_tasks()#mf_run_id=432)

# dependencies

In [ ]:
%pip list

In [ ]:
%pip list --format=freeze > requirements.txt

# END